In [ ]:
!pip install pandas numpy sklearn xgboost torch transformers nltk tensorflow

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install scikit-learn

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Fake.csv to Fake.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving True.csv to True.csv


In [ ]:
import pandas as pd

df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the 'punkt_tab' data package

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import torch
import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Download stopwords
nltk.download('stopwords')
nltk.download('punkt')

# Load Dataset (Modify the path accordingly)
df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

# Add labels (1 = Real, 0 = Fake)
df_fake["label"] = 0
df_real["label"] = 1

# Merge both datasets
df = pd.concat([df_fake, df_real], axis=0).reset_index(drop=True)

# Drop unnecessary columns if present
df = df[['title', 'text', 'label']]

# Preprocessing function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", '', text)
    text = re.sub(r"<.*?>", '', text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)

# Apply cleaning function
df["text"] = df["text"].apply(clean_text)

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.3, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Models to evaluate
models = {
    "Naïve Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(kernel="linear"),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

# Training and Evaluating Models
results = {}

for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results[name] = [accuracy, precision, recall, f1]
    print(f"{name}: Accuracy = {accuracy:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1-Score = {f1:.4f}")

# Convert results to DataFrame
results_df = pd.DataFrame(results, index=["Accuracy", "Precision", "Recall", "F1-Score"]).T
print("\nFinal Model Comparison:\n", results_df)

# LSTM Model (Deep Learning Approach)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

# Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 500
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding="post")

# LSTM Model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_length),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train_pad, y_train, epochs=3, batch_size=64, validation_data=(X_test_pad, y_test))

# Evaluate LSTM
y_pred_lstm = (model.predict(X_test_pad) > 0.5).astype("int32").flatten()
lstm_accuracy = accuracy_score(y_test, y_pred_lstm)
lstm_precision = precision_score(y_test, y_pred_lstm)
lstm_recall = recall_score(y_test, y_pred_lstm)
lstm_f1 = f1_score(y_test, y_pred_lstm)

results["LSTM"] = [lstm_accuracy, lstm_precision, lstm_recall, lstm_f1]

# BERT Model (Deep Learning Approach)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenizing and preparing dataset
def bert_tokenize(sentences, labels, max_length=512):
    inputs = tokenizer(sentences.tolist(), max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], torch.tensor(labels.tolist()))
    return dataset

train_dataset = bert_tokenize(X_train, y_train)
test_dataset = bert_tokenize(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)

# Training BERT (for simplicity, only 1 epoch)
bert_model.train()
for batch in train_dataloader:
    optimizer.zero_grad()
    input_ids, attention_mask, labels = batch
    outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluate BERT
bert_model.eval()
y_pred_bert = []
for batch in test_dataloader:
    input_ids, attention_mask, _ = batch
    with torch.no_grad():
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        y_pred_bert.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())

bert_accuracy = accuracy_score(y_test, y_pred_bert)
bert_precision = precision_score(y_test, y_pred_bert)
bert_recall = recall_score(y_test, y_pred_bert)
bert_f1 = f1_score(y_test, y_pred_bert)

results["BERT"] = [bert_accuracy, bert_precision, bert_recall, bert_f1]

# Final comparison table
results_df = pd.DataFrame(results, index=["Accuracy", "Precision", "Recall", "F1-Score"]).T
print("\nFinal Model Comparison:\n", results_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Naïve Bayes: Accuracy = 0.8895, Precision = 0.8804, Recall = 0.8956, F1-Score = 0.8879
Logistic Regression: Accuracy = 0.9179, Precision = 0.9206, Recall = 0.9107, F1-Score = 0.9156
Random Forest: Accuracy = 0.9085, Precision = 0.8986, Recall = 0.9160, F1-Score = 0.9072
SVM: Accuracy = 0.9269, Precision = 0.9256, Recall = 0.9247, F1-Score = 0.9251


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:35:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost: Accuracy = 0.9248, Precision = 0.9253, Recall = 0.9203, F1-Score = 0.9228

Final Model Comparison:
                      Accuracy  Precision    Recall  F1-Score
Naïve Bayes          0.889532   0.880423  0.895587  0.887940
Logistic Regression  0.917938   0.920566  0.910657  0.915584
Random Forest        0.908469   0.898627  0.916039  0.907249
SVM                  0.926881   0.925647  0.924650  0.925148
XGBoost              0.924776   0.925325  0.920344  0.922828
Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
